In [1]:
import sys,os
sys.path.append("/home/panayot/Documents/site_similarity")

In [2]:
from utils.notebook_utils import (combined_nodes_referral_sites_audience_overlap,
    get_alexa_information_sections, 
    extract_node_features)

In [3]:
combined_data_corpus_2020_level_0_1 = combined_nodes_referral_sites_audience_overlap(data_year='2020', level=1)

For file "corpus_2020_referral_sites.json" -> load 859 records
For file "corpus_2020_referral_sites_level_1.json" -> load 2333 records
Loaded 3489 nodes with records level <= 1 and child size:16981
referral_sites node size: 11371 audience_overlap node size: 17010


In [4]:
len(combined_data_corpus_2020_level_0_1)

28381

In [5]:
unique_sites_corpus_2020_level_0_1 = set()

for source, target in combined_data_corpus_2020_level_0_1:
    unique_sites_corpus_2020_level_0_1.add(source)
    unique_sites_corpus_2020_level_0_1.add(target)

print(len(unique_sites_corpus_2020_level_0_1))

12303


In [6]:
import redis as r

redis = r.Redis()

len([el for el in unique_sites_corpus_2020_level_0_1 if not redis.get(el)])

0

In [7]:
alexa_data_for_nodes = get_alexa_information_sections(unique_sites_corpus_2020_level_0_1, specific_section=None)

In [8]:
a = dict(zip(unique_sites_corpus_2020_level_0_1, alexa_data_for_nodes))

In [9]:
len(a)

12303

In [10]:
a['sonorannews.com']['alexa_rank']

{'site_rank': '# 1,490,554',
 'site_rank_over_past_90_days': '532 K',
 'three_month_rank_data': {},
 'country_alexa_ranks': []}

In [11]:
a['curvetube.com']

{'comparison_metrics': None,
 'similar_sites_by_audience_overlap': [{'site': 'solarconflict.com',
   'overlap_score': '13.3'},
  {'site': 'dobmovies.com', 'overlap_score': '9.0'},
  {'site': 'cyberspaceandtime.com', 'overlap_score': '7.7'},
  {'site': 'upschannel.eu', 'overlap_score': '7.6'},
  {'site': 'i1os.com', 'overlap_score': '7.4'}],
 'top_industry_topics_by_social_engagement': [],
 'top_keywords_by_traffic': {'vinespeaker ratha slain': '1.14%',
  'entel pcs zte iso modem 4shared': '0.43%',
  'midnight city - m83 / ssup choreography 2 years ago': '0.39%',
  'gerben en matthea': '0.32%',
  'grubee tightcase pull start install': '0.32%'},
 'alexa_rank_90_days_trends': {'alexa_rank': '#2,124,636',
  'time_on_site': None},
 'keyword_gaps': [{'keyword': 'ホームページ開設',
   'avg_traffic_to_competitord': '33',
   'search_popularity': '33'},
  {'keyword': 'ウェブサイト開設',
   'avg_traffic_to_competitord': '33',
   'search_popularity': '28'},
  {'keyword': 'hpできました',
   'avg_traffic_to_competitord'

In [12]:
[k for k, v in a.items() if not v.get('alexa_rank')][:10]

[]

In [13]:
df_indexs, node_features = extract_node_features(a)

print('df_indexs', len(df_indexs))
print('alexa_ranks', len(node_features['alexa_ranks']))
print('daily_pageviews_per_visitors', len(node_features['daily_pageviews_per_visitors']))
print('daily_time_on_sites', len(node_features['daily_time_on_sites']))
print('total_sites_linking_ins', len(node_features['total_sites_linking_ins']))
print('bounce_rate', len(node_features['bounce_rate']))

In [19]:
import pandas as pd

node_features_df = pd.DataFrame(node_features, index=df_indexs)

node_features_df.head()

,alexa_ranks,daily_pageviews_per_visitors,daily_time_on_sites,total_sites_linking_ins,bounce_rate
gradescope.com,11014.0,4.7,296.0,103.0,0.222
parentlink.net,151438.0,3.0,203.0,93.0,0.301
nationalpartnership.org,604522.0,1.3,156.0,811.0,0.765
sharondraper.com,1209734.0,2.0,109.0,209.0,0.615
trade.gov,55944.0,1.9,137.0,2392.0,0.654


In [20]:
node_features_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12303 entries, gradescope.com to growveg.com
Data columns (total 5 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   alexa_ranks                   9128 non-null   float64
 1   daily_pageviews_per_visitors  9129 non-null   float64
 2   daily_time_on_sites           6780 non-null   float64
 3   total_sites_linking_ins       11966 non-null  float64
 4   bounce_rate                   6300 non-null   float64
dtypes: float64(5)
memory usage: 576.7+ KB


In [24]:
combined_data_corpus_2020_level_0_1_df = pd.DataFrame(combined_data_corpus_2020_level_0_1, columns=['source', 'target'])

combined_data_corpus_2020_level_0_1_df.head()

,source,target
0,crooked.com,votesaveamerica.com
1,crooked.com,art19.com
2,crooked.com,promocodeportal.com
3,crooked.com,mediamatters.org
4,crooked.com,actblue.com


In [25]:
import stellargraph as sg

G = sg.StellarGraph(node_features_df, edges=combined_data_corpus_2020_level_0_1_df[['source', 'target']])

In [27]:
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 12303, Edges: 28381

 Node types:
  default: [12303]
    Features: float32 vector, length 5
    Edge types: default-default->default

 Edge types:
    default-default->default: [28381]
        Weights: all 1 (default)
        Features: none


In [28]:
node_features_df.to_csv('corpus_2020_audience_overlap_level_0_and_1_node_features.csv')
combined_data_corpus_2020_level_0_1_df[['source', 'target']].to_csv('combined_data_corpus_2020_level_0_1_df_edges.csv', index=False)